In [11]:
import undetected_chromedriver as uc 
import os
import pandas as pd
from bs4 import BeautifulSoup
import time
from selenium.webdriver.common.by import By

In [3]:
""" Test avec Seloger"""
pages = []


## Set Chrome Options
options = uc.ChromeOptions()
#options.headless=True
#options.add_argument('--headless')
#options.add_argument(f'--proxy-server={PROXY}')
#options.add_argument(f"user-agent={my_user_agent}")

## Create Undetected Chromedriver with Proxy
driver = uc.Chrome(options=options)


url = "https://www.seloger.com/list.htm?projects=1&types=2,1&places=[{%22inseeCodes%22:[670482]}]&mandatorycommodities=0&privateseller=1&enterprise=0&qsVersion=1.0&m=search_refine-redirection-search_results"
## Send Request
driver.get(url)
annonces = driver.find_elements(By.CSS_SELECTOR, '[data-testid="sl.explore.coveringLink"]')
liste_liens = []
# Récupérez les liens des annonces
for lien in annonces:
    lien_annonce = lien.get_attribute('href')
    liste_liens.append(lien_annonce)
    print(lien_annonce)

driver.quit()



https://www.seloger.com/annonces/locations/appartement/strasbourg-67/koenigshoffen-ouest-centre-est/196415433.htm?projects=1&types=2,1&places=[{%22inseeCodes%22:[670482]}]&mandatorycommodities=0&privateseller=1&enterprise=0&qsVersion=1.0&m=search_to_detail&lv=S
https://www.seloger.com/annonces/locations/appartement/strasbourg-67/esplanade-nord-est/209798709.htm?projects=1&types=2,1&places=[{%22inseeCodes%22:[670482]}]&mandatorycommodities=0&privateseller=1&enterprise=0&qsVersion=1.0&m=search_to_detail&lv=S
https://www.seloger.com/annonces/locations/appartement/strasbourg-67/kable-nord/213792891.htm?projects=1&types=2,1&places=[{%22inseeCodes%22:[670482]}]&mandatorycommodities=0&privateseller=1&enterprise=0&qsVersion=1.0&m=search_to_detail&lv=S
https://www.seloger.com/annonces/locations/appartement/strasbourg-67/cronenbourg-est-centre-est/214331275.htm?projects=1&types=2,1&places=[{%22inseeCodes%22:[670482]}]&mandatorycommodities=0&privateseller=1&enterprise=0&qsVersion=1.0&m=search_to_

In [4]:
liste_liens[3]

'https://www.seloger.com/annonces/locations/appartement/strasbourg-67/cronenbourg-est-centre-est/214331275.htm?projects=1&types=2,1&places=[{%22inseeCodes%22:[670482]}]&mandatorycommodities=0&privateseller=1&enterprise=0&qsVersion=1.0&m=search_to_detail&lv=S'

In [10]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

results = pd.DataFrame()

options = uc.ChromeOptions()
driver = uc.Chrome(options=options) 

for i in range(5):
    url_annonce = liste_liens[i]
    driver.get(url_annonce)

    try:
        # Utilisez une attente explicite pour attendre que la page accepter les cookies s'affiche
        continuer_sans_accepter_button = WebDriverWait(driver, 20).until(
            EC.presence_of_all_elements_located((By.XPATH, "//span[@class='didomi-continue-without-agreeing']"))
        )

        #Continue sans accepter les cookies
        continuer_sans_accepter_button[0].click()

        # Wait a bit for the text to fully load after clicking
        time.sleep(2)
            
    except Exception as e:
        print(f"Could not click 'Continuer sans accepter button: {e}")


    try: 
        voir_tout_button = driver.find_element(By.XPATH, "//button[@data-test='show-detail-feature-button-desktop']")
        voir_tout_button.click()
        time.sleep(1)

    except Exception as e:
        print(f"Could not click 'Voir toutes les caractéristiques' button: {e}")

    #Titre de l'annonce:
    titre_annonce = driver.find_elements(By.XPATH, '//div[@class="Summarystyled__Title-sc-1u9xobv-4 dbveQQ"]')[0].text


    #Prix:
    prix = driver.find_elements(By.XPATH, '//span[@class="global-styles__TextNoWrap-sc-1gbe8ip-6 gxurQr"]')[1].text #à voir si c'est toujours la 2e span 


    #Autres infos:
    #Ordre: Nb pièces, (Nb chambres), Surface, Nb etages, Balcon, Terasse, Jardin, Garage, Ascenceur, Piscine  ?Parking?
    autres_infos_WebElement = driver.find_elements(By.XPATH, '//div[@class="Tags__TagContainer-sc-edpl7u-0 EPxew"]')
    autres_infos = [element.text for element in autres_infos_WebElement]

    #Quartier:
    quartier = driver.find_elements(By.XPATH, '//span[@data-test="neighbourhood"]')[0].text

    #Ville + code postal:
    ville_cp = driver.find_elements(By.XPATH, '//span[@class="Localizationstyled__City-sc-gdkcr2-1 bgtLnh"]')[0].text


    #Description:
    description = driver.find_elements(By.XPATH, "//div[@class='ShowMoreText__UITextContainer-sc-1swit84-0 fDeZMv']")[0].text

    data_dict = {
        "Titre de l'annonce": [titre_annonce],
        "Prix": [prix],
        "Autres Infos": [", ".join(autres_infos)],
        "Quartier": [quartier],
        "Ville et CP": [ville_cp],
        "Description": [description]
    }


    #Caractéristiques:
    #Récupère les blocs de caractéristiques (ex: Extérieur, Cadre et situation...)
    blocs_caract = driver.find_elements(By.CSS_SELECTOR, "div.TitledDescription__TitledDescriptionContainer-sc-p0zomi-0")


    for bloc in blocs_caract:
        titre_caract_l = bloc.find_elements(By.CSS_SELECTOR, "div.feature-title") 
        if not titre_caract_l: #Vérifie si titre_caract est vide
            continue
        titre_caract = titre_caract_l[0].text #Récupère le titre des caractéristiques
        caract_list = bloc.find_elements(By.CSS_SELECTOR, "div.GeneralFeaturesstyled__TextWrapper-sc-1ia09m5-3")
        data_dict[titre_caract] = [", ".join([caract.text for caract in caract_list])]
        

    df=pd.DataFrame(data_dict)
    results = pd.concat([results,df], ignore_index=True)
    time.sleep(2)

driver.quit()









Could not click 'Voir toutes les caractéristiques' button: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//button[@data-test='show-detail-feature-button-desktop']"}
  (Session info: chrome=121.0.6167.85); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   undetected_chromedriver             0x0000000101424168 undetected_chromedriver + 4673896
1   undetected_chromedriver             0x000000010141b9c3 undetected_chromedriver + 4639171
2   undetected_chromedriver             0x000000010100ffdd undetected_chromedriver + 397277
3   undetected_chromedriver             0x000000010105bbfc undetected_chromedriver + 707580
4   undetected_chromedriver             0x000000010105bdd1 undetected_chromedriver + 708049
5   undetected_chromedriver             0x00000001010a0284 undetected_chromedriver + 987780
6   undetected_chromedriver             0x000000

IndexError: list index out of range

In [6]:
results

,Titre de l'annonce,Prix,Autres Infos,Quartier,Ville et CP,Description,Extérieur,Cadre et situation,Services et accessibilité,Hygiène,Surfaces annexes,Cuisine,Pièces à vivre
0,Appartement à louer,751 €,"2 pièces, 1 chambre, 71 m², Étage 12/–",Quartier Canardiere Ouest Est,Strasbourg (67100),Spacieux appartement 2 pièces de 71 M² sur STR...,Balcon,Belle vue,Ascenseur,Salle de bain (baignoire),NaN,NaN,NaN
1,Appartement meublé à louer,1 400 €,"3 pièces, 2 chambres, 85 m², Étage 6/–",Quartier Robertsau Centre,Strasbourg (67000),Appartement 3 pièces idéalement situé ! Au 6 è...,Terrasse,NaN,Ascenseur,Salle de bain (baignoire),Cave,Cuisine séparée équipée,"Séjour / salon, Salle à manger, Entrée séparée"
2,Appartement à louer,900 €,"3 pièces, 2 chambres, 70 m², Étage 3/–",Quartier Hautepierre Centre,Strasbourg (67200),"Bel appartement situé au Rue Alfred de Vigny, ...",Balcon,NaN,Ascenseur,NaN,Garage,NaN,"Séjour / salon, Entrée séparée, Placards / ran..."
3,Appartement meublé à louer,662 €,"2 pièces, 1 chambre, 42 m², Étage 2/–",Quartier Forêt Noire Centre,Strasbourg (67000),Appartement meublée et confortable en location...,NaN,"Sans vis-à-vis, Belle vue","Ascenseur, Digicode, Interphone",NaN,NaN,NaN,NaN
4,Appartement meublé à louer,1 450 €,"3 pièces, 2 chambres, 85 m², Étage 5/5",Quartier Vauban Est,Strasbourg (67000),"Bel appartement meublé, entièrement rénové, id...",NaN,NaN,Pas d'ascenseur,Salle d'eau (douche),NaN,Cuisine équipée,NaN


In [11]:
results = pd.DataFrame()

In [7]:
def retrieve_data(url_annonce):
    options = uc.ChromeOptions()
    driver = uc.Chrome(options=options)
    driver.get(url_annonce)

    try:
        # Utilisez une attente explicite pour attendre que la page accepter les cookies s'affiche
        continuer_sans_accepter_button = WebDriverWait(driver, 100).until(
            EC.presence_of_all_elements_located((By.XPATH, "//span[@class='didomi-continue-without-agreeing']"))
        )

        #Continue sans accepter les cookies
        continuer_sans_accepter_button[0].click()

        # Wait a bit for the text to fully load after clicking
        time.sleep(2)
            
    except Exception as e:
        print(f"Could not click 'Continuer sans accepter button: {e}")


    try: 
        voir_tout_button = driver.find_element(By.XPATH, "//button[@data-test='show-detail-feature-button-desktop']")
        voir_tout_button.click()
        time.sleep(1)

    except Exception as e:
        print(f"Could not click 'Voir toutes les caractéristiques' button: {e}")

    #Titre de l'annonce:
    titre_annonce = driver.find_elements(By.XPATH, '//div[@class="Summarystyled__Title-sc-1u9xobv-4 dbveQQ"]')[0].text


    #Prix:
    prix = driver.find_elements(By.XPATH, '//span[@class="global-styles__TextNoWrap-sc-1gbe8ip-6 gxurQr"]')[1].text #à voir si c'est toujours la 2e span 


    #Autres infos:
    #Ordre: Nb pièces, (Nb chambres), Surface, Nb etages, Balcon, Terasse, Jardin, Garage, Ascenceur, Piscine  ?Parking?
    autres_infos_WebElement = driver.find_elements(By.XPATH, '//div[@class="Tags__TagContainer-sc-edpl7u-0 EPxew"]')
    autres_infos = [element.text for element in autres_infos_WebElement]

    #Quartier:
    quartier = driver.find_elements(By.XPATH, '//span[@data-test="neighbourhood"]')[0].text

    #Ville + code postal:
    ville_cp = driver.find_elements(By.XPATH, '//span[@class="Localizationstyled__City-sc-gdkcr2-1 bgtLnh"]')[0].text


    #Description:
    description = driver.find_elements(By.XPATH, "//div[@class='ShowMoreText__UITextContainer-sc-1swit84-0 fDeZMv']")[0].text

    data_dict = {
        "Titre de l'annonce": [titre_annonce],
        "Prix": [prix],
        "Autres Infos": [", ".join(autres_infos)],
        "Quartier": [quartier],
        "Ville et CP": [ville_cp],
        "Description": [description]
    }


    #Caractéristiques:
    #Récupère les blocs de caractéristiques (ex: Extérieur, Cadre et situation...)
    blocs_caract = driver.find_elements(By.CSS_SELECTOR, "div.TitledDescription__TitledDescriptionContainer-sc-p0zomi-0")


    for bloc in blocs_caract:
        titre_caract_l = bloc.find_elements(By.CSS_SELECTOR, "div.feature-title") 
        if not titre_caract_l: #Vérifie si titre_caract est vide
            continue
        titre_caract = titre_caract_l[0].text #Récupère le titre des caractéristiques
        caract_list = bloc.find_elements(By.CSS_SELECTOR, "div.GeneralFeaturesstyled__TextWrapper-sc-1ia09m5-3")
        data_dict[titre_caract] = [", ".join([caract.text for caract in caract_list])]
        

    df2=pd.DataFrame(data_dict)
    print(df2)
    driver.quit()